This notebook is for Gromacs simulation. the input for this notebook is the output of the CHARMM-GUI (https://www.charmm-gui.org/) solution builder.

In [ ]:
!nvidia-smi

In [ ]:
# upgrade cmake; required for GROMACS 2021 or higher version
!pip install cmake --upgrade

#Installing py3Dmol using pip
!pip install py3Dmol

#Importing py3Dmol for safety
import py3Dmol

#Installing biopython using pip
!pip install biopython

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## to unzip the input generator file from CHARMM-GUI into the folder MD
!tar -xvzf /content/drive/MyDrive/Zipped_pro/'1PROD135A.tgz' -C /content/drive/MyDrive/MD/

In [ ]:
## to rename the folder which was zipped (charmm) to the protein_name (here : alrD135A)
from google.colab import drive
import os

def rename_folders_with_prefix(directory, prefix, new_name):
    # Get list of all folders in the directory
    folders = [f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f))]

    # Rename folders that start with the specified prefix
    for folder in folders:
        if folder.startswith(prefix):
            old_path = os.path.join(directory, folder)
            new_path = os.path.join(directory, new_name)
            os.rename(old_path, new_path)
            print(f"Renamed '{folder}' to '{new_name}'")


# Specify the directory path in Google Drive, prefix to match, and new name
directory_path = "/content/drive/MyDrive/MD"
prefix_to_match = "charmm"
new_folder_name = "alrD135A"

# Call the function to rename folders
rename_folders_with_prefix(directory_path, prefix_to_match, new_folder_name)


In [ ]:
##going inside the unzipped folder's gromacs dircectory
import os
%cd /content/drive/MyDrive/MD/alrD135A/gromacs/
!pwd

In [ ]:
# Creat working directory named run
%mkdir alrD135Arun
# Change to the directory "run"
%cd alrD135Arun
# Make sure you are in the right folder. This will show you the path :)
!pwd

In [ ]:
# access permission and check if gromacs works
%%bash
chmod +x "/content/drive/MyDrive/gmx/gromacs/bin/gmx"
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx -h

In [ ]:
## copy the files which are in the gromacs folder to the gromacs/run folder where
## all the files will be kept from gmx command
import os
import shutil

# Define source and destination directories
source_dir = '/content/drive/MyDrive/MD/alrD135A/gromacs'
destination_dir = '/content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun'

# Get list of files in source directory
files_list = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f))]

# Copy each file from source directory to destination directory
for file_name in files_list:
    source_file_path = os.path.join(source_dir, file_name)
    destination_file_path = os.path.join(destination_dir, file_name)
    shutil.copyfile(source_file_path, destination_file_path)

print("Files copied successfully!")

In [ ]:
#move the folder "toppar" to the run directory in toppar there are .itp files
import shutil

# Define source and destination directories
source_dir = '/content/drive/MyDrive/MD/alrD135A/gromacs/toppar'
destination_dir = '/content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun'

# Move the entire directory to the destination directory
shutil.move(source_dir, destination_dir)

print("Directory moved successfully!")

In [ ]:
##copy the .mdp files from zipped_pro folder to the run folder as all the gmx commnad looks
# for file in run folder until specified
import os
import shutil
from google.colab import drive

def copy_files_with_extension(src_directory, dest_directory, extension):
    # Get list of all files in the source directory
    files = [f for f in os.listdir(src_directory) if os.path.isfile(os.path.join(src_directory, f))]

    # Create destination directory if it doesn't exist
    if not os.path.exists(dest_directory):
        os.makedirs(dest_directory)

    # Copy files with the specified extension to the destination directory
    for file in files:
        if file.endswith(extension):
            src_path = os.path.join(src_directory, file)
            dest_path = os.path.join(dest_directory, file)
            shutil.copy(src_path, dest_path)
            print(f"Copied '{file}' to '{dest_directory}'")


# Specify the source and destination directories in Google Drive, and the file extension
source_directory = "/content/drive/MyDrive/Zipped_pro/"
destination_directory = "/content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun"
file_extension = ".mdp"

# Call the function to copy files with the specified extension
copy_files_with_extension(source_directory, destination_directory, file_extension)


In [ ]:
!pwd

In [ ]:
#revised minimization.mdp
!cat /content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun/MD_minimization.mdp

EM step it creates files like minimization(tpr,gro,trr,edr,log)

addtional information of mdrun is available at https://manual.gromacs.org/current/onlinehelp/gmx-mdrun.html

In [ ]:
# ntmpi =1 works best for all the mdrun commnad "step3_input" we get it from charmm gui.
# -pme gpu -npme 1 for increase perfomance.
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx grompp -f MD_minimization.mdp -o alrD135Aminimization.tpr -c step3_input.gro  -r step3_input.gro -p topol.top
gmx mdrun -v -deffnm alrD135Aminimization -nb gpu -ntmpi 1

get a XVG file for plot on EM per steps


In [ ]:
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
echo 12 | gmx energy -f alrD135Aminimization.edr -o alrD135Apotential.xvg -xvg none

In [ ]:
#plot the EM step and how much the potential energy is lowered.
import pandas as pd
df = pd.read_csv('alrD135Apotential.xvg', sep='\s+', header=None, names=['step','energy(kJ Mol-1)'])
df.plot('step')

In [ ]:
#revised minimization.mdp
!cat /content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun/NVT_equilibration.mdp

Isothermal-isochoric equilibration

In [ ]:
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx grompp -f NVT_equilibration.mdp -o alrD135ANVT_equilibration.tpr -c alrD135Aminimization.gro -r alrD135Aminimization.gro -p topol.top -n index.ndx
gmx mdrun -v -deffnm alrD135ANVT_equilibration -nb gpu -ntmpi 1

In [ ]:
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
echo 17 |gmx energy -f alrD135ANVT_equilibration.edr -o  alrD135A-temp.xvg -xvg none

In [ ]:
import pandas as pd
df = pd.read_csv('alrD135A-temp.xvg', sep='\s+', header=None, names=['time (ps)','temperature (K)'])
df.plot('time (ps)')

In [ ]:
#revised minimization.mdp
!cat /content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun/NPT_equilibration.mdp
!pwd

In [ ]:
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx grompp -f NPT_equilibration.mdp -o alrD135ANPT_equilibration.tpr -c alrD135ANVT_equilibration.gro -r alrD135ANVT_equilibration.gro -p topol.top -n index.ndx -t alrD135ANVT_equilibration.cpt
gmx mdrun -v -deffnm alrD135ANPT_equilibration -nb gpu -ntmpi 1

In [ ]:
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
## to get the  pressure change while NPTequilibration
echo 17 |gmx energy -f alrD135ANPT_equilibration.edr -o  alrD135A-pressure.xvg -xvg none
##to get the  density change while NPTequilibration
echo 23 |gmx energy -f alrD135ANPT_equilibration.edr -o  alrD135A-density.xvg -xvg none

In [ ]:
#revised minimization.mdp
!cat /content/drive/MyDrive/MD/alrD135A/gromacs/alrD135Arun/MD_production.mdp

In [ ]:
## command for MD production
%%bash
source "/content/drive/MyDrive/gmx/gromacs/bin/GMXRC"
gmx grompp -f MD_production.mdp -o alrD135Astep5_production.tpr -c alrD135ANPT_equilibration.gro -t alrD135ANPT_equilibration.cpt -p topol.top -n index.ndx
gmx mdrun -s alrD135Astep5_production -cpi -nb gpu -ntmpi 1

In [ ]:
##rename "traj_comp"
import os

def rename_files():
    directory = os.getcwd()
    files = os.listdir(directory)

    for filename in files:
        if filename == "traj_comp.xtc":
            new_filename = "alrD135Atraj_comp.xtc"
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f"Renamed {filename} to {new_filename}")

rename_files()



In [ ]:
import os

def rename_files():
    directory = os.getcwd()
    files = os.listdir(directory)

    for filename in files:
        if filename == "ener.edr":
            new_filename = "alrD135Aener.edr"
            os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
            print(f"Renamed {filename} to {new_filename}")

rename_files()
